# Quick start
```{warning}
It is currently under devellopement. Please contact me before using it.
```

## Load DICe input

In [1]:
adr_DICe='/data/Manips/Columnar_Ice/CI02/DIC_Analysis/1n_analysis/n16/'

In [2]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [3]:
import xarray_dic.loadDIC as ldDIC
import xarray_symTensor2d.xarray_symTensor2d as xT
import xarray_dic.xarray_dic as xd

In [4]:
ds=ldDIC.loadDICe(adr_DICe,0.01,10)

In [5]:
ds

<xarray.Dataset>
Dimensions:       (d: 2, sT: 6, time: 45, xd: 96, xt: 95, yd: 102, yt: 101)
Coordinates:
  * xd            (xd) float64 0.0 0.16 0.32 0.48 ... 14.72 14.88 15.04 15.2
  * yd            (yd) float64 0.0 0.16 0.32 0.48 ... 15.68 15.84 16.0 16.16
  * xt            (xt) float64 0.16 0.32 0.48 0.64 ... 14.72 14.88 15.04 15.2
  * yt            (yt) float64 0.16 0.32 0.48 0.64 ... 15.68 15.84 16.0 16.16
  * time          (time) float64 10.0 20.0 30.0 40.0 ... 420.0 430.0 440.0 450.0
Dimensions without coordinates: d, sT
Data variables:
    displacement  (time, yd, xd, d) float64 -0.01672 0.02898 ... 0.207 0.2193
    strain        (time, yt, xt, sT) float64 -0.0009465 0.001626 nan ... nan nan
Attributes:
    unit_time:      min
    step_size:      0.01
    unit_position:  millimeter
    window_size:    16
    path_dat:       /data/Manips/Columnar_Ice/CI02/DIC_Analysis/1n_analysis/n16/

## Visualize the data
To plot the strain field and displacement field for a given time step ts

In [6]:
ts=10

In [7]:
plt.figure(figsize=(20,10))
plt.subplot(231)
ds.strain[ts,:,:,0].plot()
plt.axis('equal')
plt.subplot(232)
ds.strain[ts,:,:,1].plot()
plt.axis('equal')
plt.subplot(233)
ds.strain[ts,:,:,3].plot()
plt.axis('equal')
plt.subplot(234)
ds.displacement[ts,:,:,0].plot()
plt.axis('equal')
plt.subplot(235)
ds.displacement[ts,:,:,1].plot()
plt.axis('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.08, 15.280000000000001, -0.08, 16.240000000000002)

## Compute the deformation equivalent

The deformation equivelent of $\varepsilon$ is given by :

$\varepsilon_{eq}=\sqrt{\frac{2}{3}\varepsilon_{ij}^d\varepsilon_{ij}^d}$

with : $\varepsilon^d=\varepsilon-\frac{1}{3}tr(\varepsilon)I$

Definition from [wikipedia](https://en.wikipedia.org/wiki/Infinitesimal_strain_theory)

In [8]:
ds['eqVonMises']=ds.strain.sT.eqVonMises()
ds['eqVM_ln']=ds.strain.sT.eqVonMises(lognorm=True)

/home/chauvet/Documents/GitToolboxs/DIC/xarray_symTensor2d/xarray_symTensor2d/xarray_symTensor2d.py:29: RuntimeWarning: Mean of empty slice
  tr=np.nanmean(np.array(self._obj[...,0:3]),axis=-1)
/home/chauvet/Documents/GitToolboxs/DIC/xarray_symTensor2d/xarray_symTensor2d/xarray_symTensor2d.py:42: RuntimeWarning: divide by zero encountered in log
  deq[i,...]=np.log(deq[i,...]/med[i])


In [9]:
plt.figure(figsize=(12,5))
plt.subplot(121)
ds.eqVonMises[ts].plot()
plt.axis('equal')
plt.subplot(122)
ds.eqVM_ln[ts].plot()
plt.axis('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.08, 15.280000000000001, 0.08, 16.240000000000002)

## Compute the macroscopic strain
### Compute the average of the strain tensor

In [10]:
ds['mean_eyy']=ds.strain.sT.mean('tyy')
ds['mean_exx']=ds.strain.sT.mean('txx')

### Compute the ```DIC_line```

The ```DIC_line``` is defined in [1, section 2.2.3.1](https://tel.archives-ouvertes.fr/tel-01624418)

In [11]:
ds['dl_x']=ds.dic.DIC_line('x')
ds['dl_y']=ds.dic.DIC_line('y')

In [14]:
plt.figure()
ds.mean_exx.plot.line('b-',label='<exx>')
ds.dl_x.plot.line('b--',label='x, DIC-line')
ds.mean_eyy.plot.line('r-',label='<eyy>')
ds.dl_y.plot.line('r--',label='y, DIC-line')
plt.grid()
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …